# Разработка собственного LSTM классификатора для чат-бота МУИВ

**Автор:** Синицин Михаил Дмитриевич  
**Тема ВКР:** Разработка интеллектуального чат-бота для автоматизации консультирования абитуриентов  
**Дата:** Январь 2026

---

## Содержание:
1. Импорт библиотек
2. Загрузка и анализ датасета
3. Обучение Word2Vec эмбеддингов
4. Архитектура LSTM модели
5. Обучение модели
6. Оценка качества и метрики
7. Тестирование на реальных вопросах

## 1. Импорт библиотек

In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

# Sklearn метрики
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix
)

# Word2Vec
from gensim.models import Word2Vec

# Проверка GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используемое устройство: {device}")
print(f"PyTorch версия: {torch.__version__}")

## 2. Загрузка и анализ датасета

In [ ]:
# Загрузка датасета
# Замените путь на актуальный
DATASET_PATH = 'ml/data/dataset.json'

with open(DATASET_PATH, 'r', encoding='utf-8') as f:
    data = json.load(f)

print(f"Загружено примеров: {len(data)}")
print(f"\nПример записи:")
print(json.dumps(data[0], indent=2, ensure_ascii=False))

In [ ]:
# Преобразование в DataFrame
df = pd.DataFrame(data)
print(f"Размер датасета: {df.shape}")
print(f"\nКолонки: {df.columns.tolist()}")
print(f"\nРаспределение по категориям:")
print(df['category'].value_counts())

In [ ]:
# Визуализация распределения классов
plt.figure(figsize=(12, 6))
category_counts = df['category'].value_counts()
colors = plt.cm.Set3(np.linspace(0, 1, len(category_counts)))

bars = plt.bar(category_counts.index, category_counts.values, color=colors, edgecolor='black')
plt.xlabel('Категория', fontsize=12)
plt.ylabel('Количество примеров', fontsize=12)
plt.title('Распределение примеров по категориям в датасете', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')

# Добавляем значения над столбцами
for bar, count in zip(bars, category_counts.values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 20, 
             str(count), ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig('distribution_categories.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nГрафик сохранён: distribution_categories.png")

In [ ]:
# Анализ длины вопросов
df['question_length'] = df['question'].apply(lambda x: len(x.split()))

print(f"Статистика длины вопросов (в словах):")
print(f"  Минимум: {df['question_length'].min()}")
print(f"  Максимум: {df['question_length'].max()}")
print(f"  Среднее: {df['question_length'].mean():.2f}")
print(f"  Медиана: {df['question_length'].median():.2f}")

plt.figure(figsize=(10, 5))
plt.hist(df['question_length'], bins=30, color='steelblue', edgecolor='black', alpha=0.7)
plt.xlabel('Длина вопроса (слов)', fontsize=12)
plt.ylabel('Частота', fontsize=12)
plt.title('Распределение длины вопросов', fontsize=14, fontweight='bold')
plt.axvline(df['question_length'].mean(), color='red', linestyle='--', label=f'Среднее: {df["question_length"].mean():.1f}')
plt.legend()
plt.tight_layout()
plt.savefig('question_length_distribution.png', dpi=150)
plt.show()

## 3. Предобработка текста и Word2Vec эмбеддинги

In [ ]:
def preprocess_text(text):
    """Предобработка текста: приведение к нижнему регистру, удаление спецсимволов"""
    text = text.lower()
    text = re.sub(r'[^а-яёa-z0-9\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def tokenize(text):
    """Токенизация текста"""
    return preprocess_text(text).split()

# Применяем предобработку
df['tokens'] = df['question'].apply(tokenize)

print("Примеры токенизации:")
for i in range(3):
    print(f"  Вопрос: {df['question'].iloc[i]}")
    print(f"  Токены: {df['tokens'].iloc[i]}")
    print()

In [ ]:
# Обучение Word2Vec эмбеддингов
print("Обучение Word2Vec модели...")

EMBEDDING_DIM = 100
WINDOW_SIZE = 5
MIN_COUNT = 2

sentences = df['tokens'].tolist()

w2v_model = Word2Vec(
    sentences=sentences,
    vector_size=EMBEDDING_DIM,
    window=WINDOW_SIZE,
    min_count=MIN_COUNT,
    workers=4,
    epochs=10,
    sg=1  # Skip-gram
)

print(f"\nWord2Vec модель обучена!")
print(f"  Размер словаря: {len(w2v_model.wv)}")
print(f"  Размерность эмбеддингов: {EMBEDDING_DIM}")

# Примеры похожих слов
print(f"\nСлова, похожие на 'документы':")
for word, score in w2v_model.wv.most_similar('документы', topn=5):
    print(f"  {word}: {score:.4f}")

In [ ]:
# Создание словаря и матрицы эмбеддингов
word2idx = {'<PAD>': 0, '<UNK>': 1}
for word in w2v_model.wv.key_to_index:
    word2idx[word] = len(word2idx)

vocab_size = len(word2idx)
print(f"Размер словаря (с PAD и UNK): {vocab_size}")

# Создание матрицы эмбеддингов
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, idx in word2idx.items():
    if word in w2v_model.wv:
        embedding_matrix[idx] = w2v_model.wv[word]
    elif word == '<UNK>':
        embedding_matrix[idx] = np.random.normal(0, 0.1, EMBEDDING_DIM)

print(f"Размер матрицы эмбеддингов: {embedding_matrix.shape}")

## 4. Подготовка данных для обучения

In [ ]:
# Создание маппинга категорий
categories = sorted(df['category'].unique())
cat2idx = {cat: idx for idx, cat in enumerate(categories)}
idx2cat = {idx: cat for cat, idx in cat2idx.items()}

print("Маппинг категорий:")
for cat, idx in cat2idx.items():
    print(f"  {idx}: {cat}")

NUM_CLASSES = len(categories)
print(f"\nКоличество классов: {NUM_CLASSES}")

In [ ]:
# Преобразование токенов в индексы
def tokens_to_indices(tokens, word2idx, max_len=50):
    indices = [word2idx.get(token, word2idx['<UNK>']) for token in tokens]
    if len(indices) > max_len:
        indices = indices[:max_len]
    return indices

MAX_LEN = 50
df['indices'] = df['tokens'].apply(lambda x: tokens_to_indices(x, word2idx, MAX_LEN))
df['label'] = df['category'].map(cat2idx)

print(f"Максимальная длина последовательности: {MAX_LEN}")

In [ ]:
# Разделение на train/val/test
X = df['indices'].tolist()
y = df['label'].tolist()

# 70% train, 15% val, 15% test
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp
)

print(f"Размеры выборок:")
print(f"  Train: {len(X_train)} ({len(X_train)/len(X)*100:.1f}%)")
print(f"  Validation: {len(X_val)} ({len(X_val)/len(X)*100:.1f}%)")
print(f"  Test: {len(X_test)} ({len(X_test)/len(X)*100:.1f}%)")

In [ ]:
# Вычисление весов классов для борьбы с дисбалансом
class_counts = Counter(y_train)
total_samples = len(y_train)
class_weights = {cls: total_samples / (NUM_CLASSES * count) for cls, count in class_counts.items()}
class_weights_tensor = torch.tensor([class_weights[i] for i in range(NUM_CLASSES)], dtype=torch.float32).to(device)

print("Веса классов:")
for idx, weight in enumerate(class_weights_tensor):
    print(f"  {idx2cat[idx]}: {weight:.4f}")

In [ ]:
# Dataset и DataLoader
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        return torch.tensor(self.texts[idx], dtype=torch.long), torch.tensor(self.labels[idx], dtype=torch.long)

def collate_fn(batch):
    texts, labels = zip(*batch)
    texts_padded = pad_sequence(texts, batch_first=True, padding_value=0)
    labels = torch.stack(labels)
    return texts_padded, labels

BATCH_SIZE = 32

train_dataset = TextDataset(X_train, y_train)
val_dataset = TextDataset(X_val, y_val)
test_dataset = TextDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

print(f"Батчей в train: {len(train_loader)}")
print(f"Батчей в val: {len(val_loader)}")
print(f"Батчей в test: {len(test_loader)}")

## 5. Архитектура LSTM модели

In [ ]:
class MultiHeadAttention(nn.Module):
    """Multi-Head Self-Attention механизм"""
    def __init__(self, hidden_dim, num_heads=4):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = hidden_dim // num_heads
        
        self.query = nn.Linear(hidden_dim, hidden_dim)
        self.key = nn.Linear(hidden_dim, hidden_dim)
        self.value = nn.Linear(hidden_dim, hidden_dim)
        self.out = nn.Linear(hidden_dim, hidden_dim)
        
    def forward(self, x):
        batch_size, seq_len, hidden_dim = x.size()
        
        Q = self.query(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.key(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.value(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        
        scores = torch.matmul(Q, K.transpose(-2, -1)) / np.sqrt(self.head_dim)
        attn_weights = torch.softmax(scores, dim=-1)
        
        context = torch.matmul(attn_weights, V)
        context = context.transpose(1, 2).contiguous().view(batch_size, seq_len, hidden_dim)
        
        return self.out(context), attn_weights

In [ ]:
class LSTMClassifier(nn.Module):
    """
    Собственная LSTM модель для классификации вопросов абитуриентов
    
    Архитектура:
    1. Embedding Layer (100-мерные Word2Vec эмбеддинги)
    2. Bidirectional LSTM (2 слоя, hidden_size=256)
    3. Multi-Head Self-Attention (4 головы)
    4. Classifier (4-слойный MLP: 512→256→128→64→8)
    """
    
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes, 
                 num_layers=2, num_heads=4, dropout=0.4, embedding_matrix=None):
        super().__init__()
        
        # 1. Embedding Layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        if embedding_matrix is not None:
            self.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix).float())
        
        # 2. Bidirectional LSTM
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # 3. Multi-Head Self-Attention
        self.attention = MultiHeadAttention(hidden_dim * 2, num_heads)
        
        # 4. Classifier (4-слойный MLP)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim * 2, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, num_classes)
        )
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        # x: (batch_size, seq_len)
        
        # 1. Embedding
        embedded = self.dropout(self.embedding(x))  # (batch_size, seq_len, embedding_dim)
        
        # 2. BiLSTM
        lstm_out, _ = self.lstm(embedded)  # (batch_size, seq_len, hidden_dim * 2)
        
        # 3. Attention
        attn_out, attn_weights = self.attention(lstm_out)  # (batch_size, seq_len, hidden_dim * 2)
        
        # Global Average Pooling
        pooled = torch.mean(attn_out, dim=1)  # (batch_size, hidden_dim * 2)
        
        # 4. Classifier
        logits = self.classifier(pooled)  # (batch_size, num_classes)
        
        return logits

# Создание модели
HIDDEN_DIM = 256
NUM_LAYERS = 2
NUM_HEADS = 4
DROPOUT = 0.4

model = LSTMClassifier(
    vocab_size=vocab_size,
    embedding_dim=EMBEDDING_DIM,
    hidden_dim=HIDDEN_DIM,
    num_classes=NUM_CLASSES,
    num_layers=NUM_LAYERS,
    num_heads=NUM_HEADS,
    dropout=DROPOUT,
    embedding_matrix=embedding_matrix
).to(device)

# Подсчёт параметров
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("=" * 60)
print("АРХИТЕКТУРА МОДЕЛИ")
print("=" * 60)
print(model)
print("=" * 60)
print(f"Общее количество параметров: {total_params:,}")
print(f"Обучаемых параметров: {trainable_params:,}")
print(f"Размер модели: ~{total_params * 4 / 1024 / 1024:.2f} МБ")
print("=" * 60)

## 6. Обучение модели

In [ ]:
# Гиперпараметры обучения
LEARNING_RATE = 0.0005
WEIGHT_DECAY = 0.01
NUM_EPOCHS = 25
LABEL_SMOOTHING = 0.1
PATIENCE = 7  # Early stopping

# Loss function с Label Smoothing и весами классов
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor, label_smoothing=LABEL_SMOOTHING)

# Оптимизатор AdamW
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# Learning rate scheduler (Cosine Annealing)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

print("Гиперпараметры обучения:")
print(f"  Learning Rate: {LEARNING_RATE}")
print(f"  Weight Decay: {WEIGHT_DECAY}")
print(f"  Epochs: {NUM_EPOCHS}")
print(f"  Label Smoothing: {LABEL_SMOOTHING}")
print(f"  Early Stopping Patience: {PATIENCE}")
print(f"  Batch Size: {BATCH_SIZE}")

In [ ]:
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    for texts, labels in dataloader:
        texts, labels = texts.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    return total_loss / len(dataloader), accuracy

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for texts, labels in dataloader:
            texts, labels = texts.to(device), labels.to(device)
            outputs = model(texts)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    return total_loss / len(dataloader), accuracy, all_preds, all_labels

In [ ]:
# Обучение модели
print("=" * 70)
print("НАЧАЛО ОБУЧЕНИЯ")
print("=" * 70)

history = {
    'train_loss': [], 'train_acc': [],
    'val_loss': [], 'val_acc': []
}

best_val_acc = 0
patience_counter = 0
best_model_state = None

for epoch in range(NUM_EPOCHS):
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # Validate
    val_loss, val_acc, _, _ = evaluate(model, val_loader, criterion, device)
    
    # Update scheduler
    scheduler.step()
    current_lr = scheduler.get_last_lr()[0]
    
    # Save history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    # Print progress
    print(f"Epoch {epoch+1:02d}/{NUM_EPOCHS} | "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f} | "
          f"LR: {current_lr:.6f}")
    
    # Early stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        best_model_state = model.state_dict().copy()
        print(f"  ↑ Новый лучший результат! Val Acc: {best_val_acc:.4f}")
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print(f"\nEarly stopping на эпохе {epoch+1}")
            break

print("\n" + "=" * 70)
print("ОБУЧЕНИЕ ЗАВЕРШЕНО")
print(f"Лучшая Val Accuracy: {best_val_acc:.4f}")
print("=" * 70)

In [ ]:
# Визуализация процесса обучения
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
axes[0].plot(history['train_loss'], label='Train Loss', color='blue', linewidth=2)
axes[0].plot(history['val_loss'], label='Val Loss', color='orange', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Функция потерь (Cross-Entropy Loss)', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy
axes[1].plot(history['train_acc'], label='Train Accuracy', color='blue', linewidth=2)
axes[1].plot(history['val_acc'], label='Val Accuracy', color='orange', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy', fontsize=12)
axes[1].set_title('Точность классификации', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

print("График сохранён: training_curves.png")

## 7. Оценка качества на тестовой выборке

In [ ]:
# Загрузка лучшей модели
model.load_state_dict(best_model_state)

# Оценка на тестовой выборке
test_loss, test_acc, test_preds, test_labels = evaluate(model, test_loader, criterion, device)

print("=" * 70)
print("РЕЗУЛЬТАТЫ НА ТЕСТОВОЙ ВЫБОРКЕ")
print("=" * 70)
print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
print("=" * 70)

In [ ]:
# Детальные метрики
print("\n" + "=" * 70)
print("ДЕТАЛЬНЫЕ МЕТРИКИ ПО КЛАССАМ")
print("=" * 70)

# Classification Report
report = classification_report(
    test_labels, test_preds, 
    target_names=categories,
    digits=4
)
print(report)

# Сохранение метрик в DataFrame
report_dict = classification_report(
    test_labels, test_preds, 
    target_names=categories,
    output_dict=True
)

metrics_df = pd.DataFrame(report_dict).T
metrics_df.to_csv('classification_metrics.csv')
print("\nМетрики сохранены в: classification_metrics.csv")

In [ ]:
# Confusion Matrix
cm = confusion_matrix(test_labels, test_preds)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=categories, yticklabels=categories)
plt.xlabel('Предсказанный класс', fontsize=12)
plt.ylabel('Истинный класс', fontsize=12)
plt.title('Матрица ошибок (Confusion Matrix)', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

print("Матрица ошибок сохранена: confusion_matrix.png")

In [ ]:
# Сводная таблица метрик
print("\n" + "=" * 70)
print("СВОДНАЯ ТАБЛИЦА МЕТРИК")
print("=" * 70)

summary_metrics = {
    'Метрика': ['Accuracy', 'Macro Precision', 'Macro Recall', 'Macro F1-score', 'Weighted F1-score'],
    'Значение': [
        f"{accuracy_score(test_labels, test_preds):.4f}",
        f"{precision_score(test_labels, test_preds, average='macro'):.4f}",
        f"{recall_score(test_labels, test_preds, average='macro'):.4f}",
        f"{f1_score(test_labels, test_preds, average='macro'):.4f}",
        f"{f1_score(test_labels, test_preds, average='weighted'):.4f}"
    ]
}

summary_df = pd.DataFrame(summary_metrics)
print(summary_df.to_string(index=False))
summary_df.to_csv('summary_metrics.csv', index=False)
print("\nСводные метрики сохранены: summary_metrics.csv")

## 8. Тестирование на реальных вопросах

In [ ]:
def predict(model, text, word2idx, idx2cat, device):
    """Предсказание категории для одного вопроса"""
    model.eval()
    tokens = tokenize(text)
    indices = tokens_to_indices(tokens, word2idx, MAX_LEN)
    
    with torch.no_grad():
        input_tensor = torch.tensor([indices], dtype=torch.long).to(device)
        outputs = model(input_tensor)
        probs = torch.softmax(outputs, dim=1)
        pred_idx = torch.argmax(probs, dim=1).item()
        confidence = probs[0][pred_idx].item()
    
    return idx2cat[pred_idx], confidence

In [ ]:
# Реальные тестовые вопросы
real_questions = [
    ("Сколько денег надо платить за год?", "Стоимость"),
    ("Закончил колледж, могу без ЕГЭ?", "Без ЕГЭ"),
    ("Есть места в общаге?", "Общежитие"),
    ("Какие специальности на заочке?", "Формы обучения"),
    ("Проходной балл на бюджет", "Бюджет"),
    ("Как с вами связаться?", "Контакты"),
    ("Можно учиться онлайн?", "Формы обучения"),
    ("Когда подавать документы?", "Поступление"),
    ("Сколько стоит экономика?", "Стоимость"),
    ("Есть ли магистратура?", "Обучение"),
    ("Чё, реально заочно без ЕГЭ?", "Без ЕГЭ"),
    ("Слуш, а общага далеко?", "Общежитие"),
    ("Короче, сколько платить?", "Стоимость"),
    ("Какие документы нужны для поступления?", "Поступление"),
]

print("=" * 90)
print("ТЕСТИРОВАНИЕ НА РЕАЛЬНЫХ ВОПРОСАХ АБИТУРИЕНТОВ")
print("=" * 90)
print(f"{'Вопрос':<45} {'Ожидаемое':<18} {'Предсказание':<18} {'Уверенность':<12} {'Результат'}")
print("-" * 90)

correct = 0
results = []

for question, expected in real_questions:
    predicted, confidence = predict(model, question, word2idx, idx2cat, device)
    is_correct = predicted == expected
    correct += is_correct
    result = "✅" if is_correct else "❌"
    
    results.append({
        'Вопрос': question,
        'Ожидаемое': expected,
        'Предсказание': predicted,
        'Уверенность': f"{confidence*100:.1f}%",
        'Верно': is_correct
    })
    
    print(f"{question:<45} {expected:<18} {predicted:<18} {confidence*100:>6.1f}%      {result}")

print("-" * 90)
real_accuracy = correct / len(real_questions)
print(f"\nТочность на реальных вопросах: {correct}/{len(real_questions)} ({real_accuracy*100:.1f}%)")
print("=" * 90)

# Сохранение результатов
results_df = pd.DataFrame(results)
results_df.to_csv('real_questions_results.csv', index=False)
print("\nРезультаты сохранены: real_questions_results.csv")

## 9. Сохранение модели

In [ ]:
import os

# Создание директории для модели
MODEL_DIR = 'lstm_classifier_model'
os.makedirs(MODEL_DIR, exist_ok=True)

# Сохранение модели
torch.save({
    'model_state_dict': best_model_state,
    'vocab_size': vocab_size,
    'embedding_dim': EMBEDDING_DIM,
    'hidden_dim': HIDDEN_DIM,
    'num_classes': NUM_CLASSES,
    'num_layers': NUM_LAYERS,
    'num_heads': NUM_HEADS,
    'dropout': DROPOUT,
    'word2idx': word2idx,
    'idx2cat': idx2cat,
    'cat2idx': cat2idx,
    'max_len': MAX_LEN,
    'test_accuracy': test_acc,
    'best_val_accuracy': best_val_acc
}, f'{MODEL_DIR}/classifier.pt')

# Сохранение Word2Vec модели
w2v_model.save(f'{MODEL_DIR}/word2vec.model')

# Сохранение конфигурации
config = {
    'vocab_size': vocab_size,
    'embedding_dim': EMBEDDING_DIM,
    'hidden_dim': HIDDEN_DIM,
    'num_classes': NUM_CLASSES,
    'num_layers': NUM_LAYERS,
    'num_heads': NUM_HEADS,
    'dropout': DROPOUT,
    'max_len': MAX_LEN,
    'categories': categories,
    'test_accuracy': float(test_acc),
    'best_val_accuracy': float(best_val_acc)
}

with open(f'{MODEL_DIR}/config.json', 'w', encoding='utf-8') as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

print(f"Модель сохранена в директорию: {MODEL_DIR}/")
print(f"  - classifier.pt (~{os.path.getsize(f'{MODEL_DIR}/classifier.pt') / 1024 / 1024:.2f} МБ)")
print(f"  - word2vec.model")
print(f"  - config.json")

## 10. Итоговые результаты

In [ ]:
print("\n" + "=" * 70)
print("ИТОГОВЫЕ РЕЗУЛЬТАТЫ ОБУЧЕНИЯ LSTM КЛАССИФИКАТОРА")
print("=" * 70)
print(f"""
АРХИТЕКТУРА МОДЕЛИ:
  • Embedding: {vocab_size:,} слов × {EMBEDDING_DIM} dim (Word2Vec Skip-gram)
  • BiLSTM: {NUM_LAYERS} слоя × {HIDDEN_DIM} hidden × 2 (bidirectional)
  • Attention: Multi-Head Self-Attention ({NUM_HEADS} голов)
  • Classifier: MLP (512→256→128→64→{NUM_CLASSES})
  • Dropout: {DROPOUT}
  • Всего параметров: {total_params:,}

ДАННЫЕ:
  • Датасет: {len(df):,} примеров
  • Категорий: {NUM_CLASSES}
  • Train: {len(X_train):,} | Val: {len(X_val):,} | Test: {len(X_test):,}

ГИПЕРПАРАМЕТРЫ:
  • Learning Rate: {LEARNING_RATE}
  • Weight Decay: {WEIGHT_DECAY}
  • Batch Size: {BATCH_SIZE}
  • Label Smoothing: {LABEL_SMOOTHING}
  • Оптимизатор: AdamW + CosineAnnealingLR

МЕТРИКИ НА ТЕСТОВОЙ ВЫБОРКЕ:
  • Test Accuracy:     {test_acc:.4f} ({test_acc*100:.2f}%)
  • Macro Precision:   {precision_score(test_labels, test_preds, average='macro'):.4f}
  • Macro Recall:      {recall_score(test_labels, test_preds, average='macro'):.4f}
  • Macro F1-score:    {f1_score(test_labels, test_preds, average='macro'):.4f}
  • Weighted F1-score: {f1_score(test_labels, test_preds, average='weighted'):.4f}

ТЕСТИРОВАНИЕ НА РЕАЛЬНЫХ ВОПРОСАХ:
  • Точность: {correct}/{len(real_questions)} ({real_accuracy*100:.1f}%)

СОХРАНЁННЫЕ ФАЙЛЫ:
  • distribution_categories.png - распределение классов
  • training_curves.png - кривые обучения
  • confusion_matrix.png - матрица ошибок
  • classification_metrics.csv - метрики по классам
  • summary_metrics.csv - сводные метрики
  • real_questions_results.csv - результаты тестов
  • {MODEL_DIR}/ - сохранённая модель
""")
print("=" * 70)
print("\n✅ LSTM классификатор успешно обучен и готов к интеграции!")